---
title: Handling Mathematical Expressions
format:
  html: 
    toc: true
    warning: false
    error: false
code-line-numbers: true
---

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, "/app/src")

*simulib* allows one to easily create *sympy*-backed symbolic expressions from common Python types. These expressions can then be used to define expressions such as the derivative of a metabolite concentration. `DynamicExpression` is a dataclass that represents a single expression and is used for this purpose

In [2]:
from simulib.entities.dynamic import DynamicExpression
from sympy import Symbol

m, x, b = (Symbol(name) for name in ["m", "x", "b"])

sample_expression = DynamicExpression(
    expr = m*x + b
)

display(sample_expression)

DynamicExpression(expr=b + m*x, cond=None)

In [3]:
from simulib.entities.dynamic import DynamicExpression
from sympy import Symbol

m, x, b = (Symbol(name) for name in ["m", "x", "b"])

sample_expression = DynamicExpression(
    expr = m*x + b
)

display(sample_expression)

DynamicExpression(expr=b + m*x, cond=None)

In [4]:
from simulib.entities.dynamic import DynamicExpression
from sympy import Symbol

m, x, b = (Symbol(name) for name in ["m", "x", "b"])

sample_expression = DynamicExpression(
    expr = m*x + b
)

display(sample_expression)

DynamicExpression(expr=b + m*x, cond=None)

In [5]:
from simulib.entities.dynamic import DynamicExpression
from sympy import Symbol

m, x, b = (Symbol(name) for name in ["m", "x", "b"])

sample_expression = DynamicExpression(
    expr = m*x + b
)

display(sample_expression)

DynamicExpression(expr=b + m*x, cond=None)

In [6]:
from simulib.entities.dynamic import DynamicExpression
from sympy import Symbol

m, x, b = (Symbol(name) for name in ["m", "x", "b"])

sample_expression = DynamicExpression(
    expr = m*x + b
)

display(sample_expression)

DynamicExpression(expr=b + m*x, cond=None)

In [7]:
from simulib.entities.dynamic import DynamicExpression
from sympy import Symbol

m, x, b = (Symbol(name) for name in ["m", "x", "b"])

sample_expression = DynamicExpression(
    expr = m*x + b
)

display(sample_expression)

DynamicExpression(expr=b + m*x, cond=None)

In [8]:
from simulib.entities.dynamic import DynamicExpression
from sympy import Symbol

m, x, b = (Symbol(name) for name in ["m", "x", "b"])

sample_expression = DynamicExpression(
    expr = m*x + b
)

display(sample_expression)

DynamicExpression(expr=b + m*x, cond=None)

`expr` is the dataclass field that holds the desired expression. Additionally, the `cond` field can also store a relational expression to be used as the condition for `expr` to be used.

In [9]:
sample_expression_with_cond = DynamicExpression(
    expr = m*x + b,
    cond = m > 1
)

display(sample_expression_with_cond)

DynamicExpression(expr=b + m*x, cond=m > 1)

`DynamicExpression` objects can be created from a vast number of types. Dictionary and string parsing allow the user to freely add any symbols to their expressions without the need for a pre-instanced `sympy.Symbol` object.

In [10]:
sample_expression_from_dict = DynamicExpression.from_object(
    {"expr": "m * x + b", "cond": "m > 1"}
)
display(sample_expression_from_dict)

sample_expression_from_string = DynamicExpression.from_object(
    "m * x + b"
)
display(sample_expression_from_string)

DynamicExpression(expr=b + m*x, cond=m > 1)

DynamicExpression(expr=b + m*x, cond=None)

`DynamicExpression` instances can be compared for equality and will return True if both instances contain equivalent expressions in both `expr` and `cond` fields.

In [11]:
display(sample_expression_from_dict == sample_expression_with_cond)
display(sample_expression == sample_expression_from_string)

True

True

Free symbols in a `DynamicExpression` can be identified using the `free_variables` property.

In [12]:
sample_expression.free_variables

{b, m, x}

Symbols in `DynamicExpression` instances can also be substituited by other symbols or values.

In [13]:
expression_to_sub = DynamicExpression(
    expr = m*x + b
)
display(expression_to_sub)

expression_to_sub.substitute(
    {"m": 2, "b": 4}
)
display(expression_to_sub)

expression_to_sub.substitute(
    {"x": 10}
)
display(expression_to_sub)

DynamicExpression(expr=b + m*x, cond=None)

DynamicExpression(expr=2*x + 4, cond=None)

DynamicExpression(expr=24, cond=None)

### Further considerations

Although `DynamicExpression` is supported by `sympy`, there are certain limitations and considerations to take into account when creating expressions. Expressions can be typed in various formats with vastly different implications.

In this first example, we define a simple expression.

In [14]:
DynamicExpression.from_object("A * B + C")

DynamicExpression(expr=A*B + C, cond=None)

When dealing with exchange bounds, we may also supply simple expressions such as the one above, but we must also supply a `cond` parameter. In this context, and when the expression is passed as a dictionary, `cond` does not affect the parsed expression.

In [15]:
DynamicExpression.from_object({"expr":"A * B + C", "cond":"D"})

DynamicExpression(expr=A*B + C, cond=D)

However, we may also require more complex functions. At the moment, `DynamicExpression` can contain piecewise functions, whose definition varies along the function's domain. In `simulib` such functions can be parsed using the following structure:
```
[
    {"expr": <expression #1>, "cond": <condition #1>},
    {"expr": <expression #2>, "cond": <condition #2>},
    <expression #3> (condition defaults to True if all of the above are not `True`) 
]
```
Note that it is critical for these dictionaries to be enclosed in a list, so that the parser recognizes this as a piecewise expression


In [16]:
list_of_dicts = [
    {"expr": "A + 1", "cond": "B <= 2"},
    {"expr": "A * 2", "cond": "B <= 3"},
    "A"
]
piecewise_expression = DynamicExpression.from_object(list_of_dicts)
display(piecewise_expression)
piecewise_expression.substitute({"B": 2.0})
display(piecewise_expression)

DynamicExpression(expr=Piecewise((A + 1, B <= 2), (2*A, B <= 3), (A, True)), cond=None)

DynamicExpression(expr=A + 1, cond=None)

There are certain limitations to this. Equality expressions (`=`) can be used as conditions, but only in isolation. The following example works.

In [17]:
another_list_of_dicts = [
    {"expr": "X**2", "cond": "b = 2"},
    {"expr": "X**3", "cond": "b = 3"},
    "A"
]

piecewise_expression_that_works = DynamicExpression.from_object(another_list_of_dicts)
display(piecewise_expression_that_works)

piecewise_expression_that_works.substitute({"b": 2})
display(piecewise_expression_that_works)

DynamicExpression(expr=Piecewise((X**2, Eq(b, 2)), (X**3, Eq(b, 3)), (A, True)), cond=None)

DynamicExpression(expr=X**2, cond=None)

However, this one does not, since it is the combination of two relational expressions. This is a `sympy` issue which we cannot fix (as of 2025).

In [18]:
list_of_dicts_not_working = [
    {"expr": "X**2", "cond": "(b=2) & (z > 0)"},
    {"expr": "X**3", "cond": "b = 3"},
    "A"
]

try:
    piecewise_expression_not_working = DynamicExpression.from_object(list_of_dicts_not_working)
    
except NameError as e:
    print(f"Caught exception: {e}")

Caught exception: name 'b' is not defined


To have it work, equality needs to be defined with `Eq`

In [19]:
list_of_dicts_working = [
    {"expr": "X**2", "cond": "Eq (b, 2) & (z > 0)"},
    {"expr": "X**3", "cond": "b = 3"},
    "A"
]

piecewise_expression_working = DynamicExpression.from_object(list_of_dicts_working)
display(piecewise_expression_working)
piecewise_expression_working.substitute({"b": 2})
display(piecewise_expression_working)

    

DynamicExpression(expr=Piecewise((X**2, Eq(b, 2) & (z > 0)), (X**3, Eq(b, 3)), (A, True)), cond=None)

DynamicExpression(expr=Piecewise((X**2, z > 0), (A, True)), cond=None)